In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser

In [2]:
# Setup the browser for scraping
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Get the latest Mars news
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')
results = soup.find_all('div', class_='list_text')
result = results[0]

news = result.find('div', class_='content_title')
news_title = news.a.text
news_para = result.find('div', class_='article_teaser_body')
news_para2 = news_para.text.split("\n")
news_desc = news_para2[0]
    
#print("Title: " + news_title)
#print("Description: " + news_desc)

In [4]:
# Get the JPL Mars Space Featured Image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = bs(html, 'lxml')

featured_image_url = ""

articles = soup.find_all('article', class_='carousel_item')

for article in articles:
    link = article.find('a')
    href = link['data-fancybox-href']
    featured_image_url = 'https://www.jpl.nasa.gov/' + href
    featured_image_title = link['data-title']

#    print(featured_image_title)
#    print(featured_image_url)

In [5]:
# Get the latest Mars Weather Report from twitter
mars_weather = ""
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

html = browser.html
soup = bs(html, "lxml")

tweets = soup.find_all('p', class_='tweet-text')

for tweet in tweets:
    tweettext = tweet.text
    if ("Sol " in tweettext):
        mars_weather = tweettext
        break
        
#print(mars_weather)

In [6]:
# Get some Mars Facts
url = "http://space-facts.com/mars"

tables = pd.read_html(url)
df = tables[0]
df.columns = ['description','value']

mars_facts = df.to_dict()
#print(mars_facts)

In [7]:
# Covert Mars Facts to an HTML table string
df.set_index('description', inplace=True)
mars_facts_table = df.to_html()
#print(mars_facts_table)

In [8]:
# Collecting Mars Hemisphere Links
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = bs(html, 'lxml')

links = soup.find_all('a', class_='itemLink product-item')
hemisphere_image_urls = []

for link in links:
    href = link['href']
    product_url = 'https://astrogeology.usgs.gov' + href
    title = link.find('h3')
    if (title):
        browser.visit(product_url)
        html = browser.html
        soup = bs(html, "lxml")
        downloads = soup.find('div', class_='downloads')
        imgLinks = downloads.find_all('a')
        for img in imgLinks:
            if (img.text == "Original"):
                img_url = img.get('href')
                hemisphere_image_urls.append({'title': title.text, 'img_url': img_url})

#print(hemisphere_image_urls)

In [11]:
# Save all scraped data to one dictionary
mars = {'news_title': news_title, 'news_desc': news_desc, 'featured_image_title': featured_image_title, \
           'featured_image_url': featured_image_url, 'mars_weather': mars_weather, 'mars_facts_table': mars_facts_table, \
           'hemisphere_image_urls': [hemisphere_image_urls]}